<h1>Introduction to Community Detection</h1>

<h3>Motivation: The Quarantine Problem</h3>

You run the IT department at a large organization in which the employees regularly let their computers get infected with viruses that spread rapidly through the network.  Once you get a report about an infection, you want to be able to contain the spread of the infection as quickly as possible by terminating as few network connections as possible.  How do you do it?

Idea: look for "communities" in the network which can be easily disconnected from one another, and hope that you can cut the connections fast enough to contain the virus to a single community.

But how do you find these communities?

Other applications:
<ul>
<li>Broadcasting problem </li>
<li>Influence optimization </li>
<li>Sensor networks </li>
</ul>

<h3>Background on Networks</h3>

We will model a network as a graph $G$ consisting of a set of vertices $V$ and a set of edges between vertices $E$.  For simplicity, assume that $G$ is unweighted and undirected.  Some notation:

<ul>
<li> The <i>degree</i> of a vertex $v \in V$ is the number of edges with $v$ as a vertex </li>
<li> The <i>volume</i> of a set $S \subseteq V$ of vertices is the sum of the degrees of its elements:
    $$vol(S) = \sum_{v \in S} deg(v)$$
<li> The <i>complement</i> of a set $S \subseteq V$ is the set $S^c$ of all vertices not in $S$ </li>
<li> The <i>boundary</i> of a set $S \subseteq V$ is the set $\partial S$ of all edges connecting a vertex in $S$ to a vertex in $S^c$
</ul>


<h3>Communities</h3>

Intuition: a community in a network is a set of vertices with many more internal connections than external connections.

Given a set $S$ of vertices, the <i>conductance</i> of $S$ is defined to be:
$$\Phi(S) = \frac{|\partial S|}{\min(vol(S), vol(S^c))}$$
(i.e. the number of boundary edges divided by the "balanced volume" of $S$)

Broadly speaking, community detection is the science of finding sets of vertices in a graph with small conductance.  (Other measures of "community-ness" are used in the literature.)  We will focus on two sub-problems:

<ul>
<li>The Partitioning Problem: find the set of vertices whose conductance is smallest (the "optimal cut") </li>
<li>The Community Recovery Problem: find a neighborhood of a prescribed collection of vertices with small conductance  </li>
</ul>

Note that the partitioning problem includes the community recovery problem as stated as a special case.  However, for extremely large networks, partitioning is generally intractible while community recovery is often possible.

Some literature on the partitioning problem ("global clustering"):
<ul>
<li>Spectral theory </li>
<li>Semidefinite programming </li>
<li>Louvain clustering </li>
</ul>
\
Some literature on the community recovery problem ("local clustering"):
<ul>
<li>Random walks / PageRank</li>
<li>Flow algorithm</li>
</ul>

<h3>Spectral Clustering (Global)</h3>

Intuition: communities correspond to low frequence waves (normal modes).  According to physics, waves correspond to eigenvectors of the Laplace operator.

The <i>degree matrix</i> of a graph with $n$ vertices is the $n \times n$ matrix $D$ whose $i$th diagonal entry is the degree of the $i$th vertex.

The <i>adjacency matrix</i> is the $n \times n$ matrix $A$ whose $(i,j)$ entry is $1$ if there is an edge between the $i$th and $j$th vertex and $0$ otherwise.

The <i>unnormalized Laplacian matrix</i> is the matrix $L = D - A$.  The <i>Laplacian</i> is the matrix $\mathcal{L} = D^{-1/2} L D^{-1/2}$

In [ ]:
G = small_graph()
write_graph(G, "net.html")

In [ ]:
print(nx.adjacency_matrix(G))

In [ ]:
print(nx.laplacian_matrix(G))

In [ ]:
print(nx.normalized_laplacian_matrix(G))

<b>Proposition:</b> The Laplacian matrix $\mathcal{L}$ of a graph $G$ has the following properties:
<ul>
<li> The eigenvalues of $\mathcal{L}$ lie in the interval $[0,2]$</li>
<li> $0$ is always an eigenvalue of $\mathcal{L}$, and the multiplicity of $0$ is equal to the number of connected components of $G$.
</ul>

In particular, $G$ is disconnected if and only if $0$ is a repeated eigenvalue.  Key idea: if $G$ is connected but the second smallest eigenvalue of the Laplacian is small, then $G$ is "almost disconnected".

<b>Spectral Clustering Algorithm</b>

Input: a graph $G$.
<ol>
<li> Compute an eigenvector $e_2$ associated to the smallest nonzero eigenvalue of the Laplacian $\mathcal{L}$ of $G$. </li>
<li> Order the vertices so that:
$$\frac{e_2(v_1)}{\sqrt{deg(v_1)}} \geq \frac{e_2(v_2)}{\sqrt{deg(v_2)}} \geq \ldots \geq \frac{e_2(v_n)}{\sqrt{deg(v_n)}}$$</li>
<li> Compute the conductance of each "level set" for this ordering:
$$\Phi_1 = \Phi(\{v_1\})$$
$$\Phi_2 = \Phi(\{v_1, v_2\})$$
$$\Phi_3 = \Phi(\{v_1, v_2, v_3\})$$
$$\ldots \Phi_n = \Phi(\{v_1, \ldots, v_n\})$$</li>
</ol>
Output: the set $\{v_1, \ldots, v_k\}$ which minimizes $\Phi_k$.

(Because of its importance in this algorithm, $e_2$ is often called the <i>Fiedler vector</i> of $G$.)

In [ ]:
G = block_graph(.3, 5)

In [ ]:
weights = nx.fiedler_vector(G, normalized = True)

for n in G.nodes():
    G.node[n]["w"] = weights[n] / math.sqrt(G.degree(n)) * 100

In [ ]:
write_graph(G, "net.html")

<h3>PageRank Clustering (Local)</h3>

Intuition: Take a starting vertex (or collection of starting vertices) as input.  Extract a community neighboring the starting vertex by looking for those other vertices which are most likely to be visited in a short random walk beginning at that vertex.

Instead of actually simulating random walks, use the <i>personalized PageRank algorithm</i>, a variation on Brin and Page's algorithm for web search.  The PageRank algorithm aggregates probabilities associated to random walks of all different lengths, with a bias toward shorter walks.

<b>Definition:</b> The <i>random walk</i> matrix of a graph $G$ is the matrix $W = D^{-1}A$.  

If $q$ is a probability vector indexed by the vertices of $G$, then the entries of the vector $qW$ (right multiplication) give the probabilities of reaching each vertex after a one step random walk (assuming the starting vertex was chosen from the distribution $q$).

<b>Definition:</b> Let $\alpha \in (0,1)$ be a "jumping constant" and let $s$ be a fixed probability vector indexed by the vertices of $G$.  The <i>PageRank</i> vector $p = pr(s, \alpha)$ is the (unique) solution to the equation:
$$p = \alpha s + (1-\alpha)pW$$

(Usually PageRank vectors are computed using the iteration $p_{k+1} = \alpha s + (1-\alpha)p_k W$)

<b>PageRank Clustering Algorithm</b>

Input: a graph $G$ and a set $S$ of vertices.
<ol>
<li> Choose a small jumping constant $\alpha$, and let $s$ be the uniform probability vector supported on $S$.  Compute the PageRank vector $p = pr(s, \alpha)$.</li>
<li> Order the vertices so that:
$$\frac{p(v_1)}{deg(v_1)} \geq \frac{p(v_2)}{deg(v_2)} \geq \ldots \geq \frac{p(v_n)}{deg(v_n)}$$</li>
<li> Compute the conductances $\Phi_1, \ldots, \Phi_n$ of the level sets for this ordering, exactly as in the spectral clustering algorithm.

Output: the set $\{v_1, \ldots, v_k\}$ which minimizes $\Phi_k$.
</ol>

In [ ]:
G = block_graph(.3, 5)
s = {n: 1 if n in range(0,1) else 0 for n in G.nodes()}

In [ ]:
weights = nx.pagerank(G, alpha = .1, personalization = s)

In [ ]:
for n in G.nodes():
    G.node[n]["w"] = weights[n] / G.degree(n) * 100000

In [ ]:
write_graph(G, "net.html")

In [ ]:
def small_graph():
    G = nx.Graph()
    G.add_edge(0,1)
    G.add_edge(0,2)
    G.add_edge(0,3)
    G.add_edge(1,3)
    
    for n in G.nodes():
        G.node[n]["w"] = 1
    
    return G

In [ ]:
import json
import math
from random import randint
import networkx as nx

def block_graph(p, m):
    #Create four blocks
    G1 = nx.binomial_graph(50, p)
    G2 = nx.binomial_graph(50, p)
    G3 = nx.binomial_graph(50, p)
    G4 = nx.binomial_graph(50, p)

    #Add them all to the same network
    G = nx.disjoint_union(G1, G2)
    G = nx.disjoint_union(G, G3)
    G = nx.disjoint_union(G, G4)
    
    #Add 'weight' attribute to every node
    for n in G.nodes():
        G.node[n]["w"] = 1

    #Add edges between blocks
    for i in range(0,m):
        G.add_edge(randint(0,49), randint(50,99))
        G.add_edge(randint(0,49), randint(100,149))
        G.add_edge(randint(0,49), randint(150,199))
        G.add_edge(randint(50,99), randint(100,149))
        G.add_edge(randint(50,99), randint(150, 199))
        G.add_edge(randint(100,149), randint(150,199))
    
    return G

def write_graph(G, file):
    #Convert to JSON for d3
    G_json = json.dumps({
        "nodes": [{"name": n, "w": G.node[n]["w"]} for n in G.nodes()],
        "links": [{"source": e[0], "target": e[1]} for e in G.edges()]
    }, indent = 4)
    
    #Write to file
    with open("template.html") as viz:
        new_viz = viz.read().replace("{{graph}}", G_json)

    with open(file, "w") as viz2:
        viz2.write(new_viz)

In [ ]:
G = block_graph(.3, 5)

In [ ]:
write_graph(G, "net.html")